In [47]:
### Benjamin Tollison ###

from decimal import Decimal

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sympy as sp
from IPython.display import Latex, Math, display
from sympy import (
    Eq,
    Function,
    Matrix,
    cos,
    cosh,
    exp,
    integrate,
    lambdify,
    pi,
    sin,
    sinh,
    symbols,
)
from sympy.solvers.pde import pdsolve
from sympy.solvers.solveset import linsolve


def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))
def format_scientific(number:float):
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]
deg2rad = np.pi/180
rad2deg = 180/np.pi

In [48]:
def PrandtlMeyerRadians(machnumber:float)->float:
  M = machnumber
  v = np.sqrt(2.4/.4)*np.arctan2(np.sqrt(.4*(M**2-1)),np.sqrt(2.4))-np.arctan2(np.sqrt(M**2-1),1)
  return v
def FindMachTwo(deflection_angle_degrees,mach_1):
  theta = deflection_angle_degrees*deg2rad
  nu2 = theta + PrandtlMeyerRadians(mach_1)
  M,gamma = symbols('M \\gamma')
  PrandtlMeyerSymbolic = sp.sqrt((gamma+1)/(gamma-1))*sp.atan(sp.sqrt((gamma-1)*(M**2-1)/(gamma+1)))-sp.atan(M**2-1)
  partial_M_PradtlMeyerSymbolic = PrandtlMeyerSymbolic.diff(M)
  step_function = lambdify(M,(PrandtlMeyerSymbolic/partial_M_PradtlMeyerSymbolic).subs({gamma:1.4}))
  display(step_function)
  f = lambdify(M,PrandtlMeyerSymbolic.subs({gamma:1.4}))
  M2_guess = 1.1
  iteration_stopper = 1000
  while abs(f(M2_guess)-nu2) > 1e-6 or abs(step_function(M2_guess)) > 1e-8:
    M2_guess = M2_guess - step_function(M2_guess)
    iteration_stopper -= 1
    if iteration_stopper == 0:
      print('Newton-Raphson did not converge')
      break
  return M2_guess

In [49]:
# Question 1
def PrandtlMeyerRadians(machnumber:float)->float:
  M = machnumber
  v = np.sqrt(2.4/.4)*np.arctan2(np.sqrt(.4*(M**2-1)),np.sqrt(2.4))-np.arctan2(np.sqrt(M**2-1),1)
  return v
displayVariable('\\nu(M_\\infty)',PrandtlMeyerRadians(999999)*rad2deg)

<IPython.core.display.Math object>

In [50]:
# Question 2
def ThetaBetaMach(gamma_of_gas,intial_theta_guess_rad,intial_beta_guess_rad,intial_mach_guess,theta_known:bool,beta_known:bool,mach_known:bool):
  theta,beta,M1,gamma = symbols('\\theta \\beta  M_1 \\gamma')
  eqn = 2*sp.cot(beta)*((M1**2*sin(beta)-1)/(M1**2*(gamma+cos(2*beta))+2)) - sp.tan(theta)
  partial_theta = lambdify(args=[theta,beta,M1],expr=eqn.diff(theta).subs(gamma,gamma_of_gas))
  partial_beta = lambdify(args=[theta,beta,M1],expr=eqn.diff(beta).subs(gamma,gamma_of_gas))
  partial_mach = lambdify(args=[theta,beta,M1],expr=eqn.diff(M1).subs(gamma,gamma_of_gas))
  f = lambdify(args=[theta,beta,M1],expr=(eqn.subs(gamma,gamma_of_gas)))
  tolerance = 1e-6
  iteration_counter = 0
  iteration_table = {'theta_i+1':[],'beta_i+1':[],'M_i+1':[],'f_i+1':[]}
  if theta_known == False:
    theta_i = intial_theta_guess_rad
    while f(theta_i,intial_beta_guess_rad,intial_mach_guess) > tolerance or  f(theta_i,intial_beta_guess_rad,intial_mach_guess) < -tolerance:
      theta_i = theta_i - f(theta_i,intial_beta_guess_rad,intial_mach_guess)/partial_theta(theta_i,intial_beta_guess_rad,intial_mach_guess)
      iteration_table['theta_i+1'].append(theta_i),iteration_table['beta_i+1'].append(intial_beta_guess_rad),iteration_table['M_i+1'].append(intial_mach_guess),iteration_table['f_i+1'].append(f(theta_i,intial_beta_guess_rad,intial_mach_guess))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('\\theta'),theta_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return theta_i
  if beta_known == False:
    beta_i = intial_beta_guess_rad
    while f(intial_theta_guess_rad,beta_i,intial_mach_guess) > tolerance or  f(intial_theta_guess_rad,beta_i,intial_mach_guess) < -tolerance:
      beta_i = beta_i - f(intial_theta_guess_rad,beta_i,intial_mach_guess)/partial_beta(intial_beta_guess_rad,beta_i,intial_mach_guess)
      iteration_table['theta_i+1'].append(intial_theta_guess_rad),iteration_table['beta_i+1'].append(beta_i),iteration_table['M_i+1'].append(intial_mach_guess),iteration_table['f_i+1'].append(f(intial_theta_guess_rad,beta_i,intial_mach_guess))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('\\beta'),beta_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return beta_i
  if mach_known == False:
    mach_i = intial_mach_guess
    while f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i) > tolerance or  f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i) < -tolerance:
      mach_i = mach_i - f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i)/partial_mach(intial_theta_guess_rad,intial_beta_guess_rad,mach_i)
      iteration_table['theta_i+1'].append(intial_theta_guess_rad),iteration_table['beta_i+1'].append(intial_beta_guess_rad),iteration_table['M_i+1'].append(mach_i),iteration_table['f_i+1'].append(f(intial_theta_guess_rad,intial_beta_guess_rad,mach_i))
      iteration_counter += 1
      if iteration_counter >=10000:
        return print('guess did not converge')
    displayEquations(symbols('M_1'),mach_i)
    display(pd.DataFrame.from_dict(iteration_table))
    return mach_i

In [51]:
ThetaBetaMach(1.4,43*deg2rad,47*deg2rad,2,True,True,False)

<IPython.core.display.Math object>

,theta_i+1,beta_i+1,M_i+1,f_i+1
0,0.750492,0.820305,3.137007,-1.666246e-01
1,0.750492,0.820305,4.298087,-5.449334e-02
2,0.750492,0.820305,5.157522,-1.188467e-02
3,0.750492,0.820305,5.466659,-9.246653e-04
4,0.750492,0.820305,5.494964,-6.680518e-06
5,0.750492,0.820305,5.495172,-3.544370e-10


5.495171662839161

In [52]:
# Question 3
ThetaBetaMach(1.4,23*deg2rad,(90-23)*deg2rad,2,True,True,False)

<IPython.core.display.Math object>

,theta_i+1,beta_i+1,M_i+1,f_i+1
0,0.401426,1.169371,1.871560,-2.093838e-03
1,0.401426,1.169371,1.876732,-3.500282e-06
2,0.401426,1.169371,1.876741,-9.837076e-12


1.8767407607511497